There is a group of about 300 people who want to help each other. The problem is that a single person can meaningfully keep track of 150 people but all 300 want to cooperate and do it fairly to each other. How can they do that?

It's sad that you will not find the problem in any school book. If you did, the initial answer would be simple. Why don't they write down who helped whom and how much. For convinience they even can use 1 help unit called hlp.

In [1]:
from dataclasses import dataclass, field
from time import time

@dataclass
class Transaction:
    recipient: str
    sender: str
    amount: float
    timestamp: float = field(default_factory=time)

block = []

Now we can just put them on the chain.

In [2]:
block.append(Transaction(
    sender='Alise', recipient='Bob', amount=2.0
))
block.append(Transaction(
    sender='Bob', recipient='Jon', amount=4.5
))

In [3]:
block

[Transaction(recipient='Bob', sender='Alise', amount=2.0, timestamp=1541953013.3726277),
 Transaction(recipient='Jon', sender='Bob', amount=4.5, timestamp=1541953013.3726985)]

This covers direct person to person cooperation. Everyone is happy. If someone tries to cooperate inderectly, things fall apart. Since anyone can update the chain, transactions can be removed. If people try to exchange units of help there is an insentive to mess with the transaction list. Hm, how can we fix it? Let's chain those blocks so transactions can only be added.

In [68]:
import hashlib

class Block:
    transactions: list
    previous_hash: str = 'coinbase'

    def __init__(self):
        self.transactions = []

    def append(self, t: Transaction):
        self.transactions.append(t)
    
    def __repr__(self):
        return 'transactions: {}\n\nprevious hash: {}'.format(self.transactions.__repr__(), self.previous_hash)
    
    def hash(self):
        return hashlib.sha256(str(self).encode()).hexdigest()
        

In [69]:
block = Block()
block.append(Transaction(
    sender='Alise', recipient='Bob', amount=2.0
))
block.append(Transaction(
    sender='Bob', recipient='Jon', amount=4.5
))

In [70]:
block

transactions: [Transaction(recipient='Bob', sender='Alise', amount=2.0, timestamp=1541954503.6024055), Transaction(recipient='Jon', sender='Bob', amount=4.5, timestamp=1541954503.6025043)]

previous hash: coinbase

In [71]:
block.hash()

'8107add94b3d0d04eea002979089aea9018deaae47f432857bcc301e2149fdde'

Hey, look at that! We got short and unique (for our purpuses) representation of our block and all the transactions in it.

In [72]:
block.hash()

'8107add94b3d0d04eea002979089aea9018deaae47f432857bcc301e2149fdde'

In [73]:
block.append(Transaction(
    sender='Jon', recipient='Bob', amount=1.0
))

In [74]:
block.hash()

'c3c84e37361404b64a397fd641bf6ed2da3c8cf3c47f2c5d3f0678e12b1bb49f'

We need to represent a block in something hashable. We don't need anything fancy at this point so `__repr__` will give us what we need. Now let's add smarter chain class.

In [89]:
import itertools


class Chain:
    blocks: list

    def __init__(self):
        """We need initial block"""
        block = Block()
        block.append(Transaction(
            sender='Alise', recipient='Alise', amount=2.0
        ))
        self.blocks = [block]

    def push(self, block: Block):
        block.previous_hash = self.blocks[-1].hash()
        self.blocks.append(block)

    @property
    def tempared(self):
        """
        It gives you possition of invalid block according to it's naighbour to the right.
        If chain looks good it returns 0.
        """
        a, b = itertools.tee(self.blocks)
        next(b, None)
        for position, pair in enumerate(zip(a, b)):
            if pair[0].hash() != pair[1].previous_hash:
                return position
        return -1

    @property
    def is_valid(self):
        if self.tempared < 0:
            return True
        return False
        

In [110]:
chain = Chain()

In [111]:
chain.blocks

[transactions: [Transaction(recipient='Alise', sender='Alise', amount=2.0, timestamp=1541955544.5083797)]
 
 previous hash: coinbase]

In [112]:
for _ in range(10):
    block = Block()
    for j in range(3):
        block.append(Transaction(sender='Bob', recipient='Jon', amount=j))
    chain.push(block)

In [113]:
chain.is_valid

True

In [114]:
borrowed = chain.blocks[0].transactions.pop()
borrowed

Transaction(recipient='Alise', sender='Alise', amount=2.0, timestamp=1541955544.5083797)

In [115]:
chain.is_valid

False

In [116]:
chain.tempared

0

Let's put borrowed transaction back.

In [117]:
chain.blocks[0].transactions.append(borrowed)

In [118]:
chain.is_valid

True

We can only mess with transactions in the last block.

In [120]:
chain.blocks[-1].append(Transaction(recipient='Alise', sender='Jon', amount=5))

In [121]:
chain.is_valid

True